#### Convert Video type to images sequence

Deal with .AVI files from timelapse data

nombre d'images : 1020

1 image par minute


In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from utils_annotation import getListImg
from utils_annotation import saveFrames

In [2]:
path = r'C:\Users\aurel\Desktop\video_oiseaux_cam2\Data\TLC00020.AVI'

#Takes around 11s for 1 file
frames, count = getListImg(path)

img0 = cv2.imread(r'C:\Users\aurel\Desktop\video_oiseaux_cam2\t0_TLC00020.jpg')
img1 = cv2.imread(r'C:\Users\aurel\Desktop\video_oiseaux_cam2\t1_TLC00020.jpg')
img2 = cv2.imread(r'C:\Users\aurel\Desktop\video_oiseaux_cam2\t00_TLC00020.jpg')
plt.imshow(img2)

In [ ]:
dif = img0 - img1
plt.imshow(dif)

In [ ]:
mask_dif = np.sum(dif, axis=2)/3
plt.imshow(mask_dif.astype('uint'), cmap='gray')

In [ ]:
import tensorflow as tf
model_dir = r'C:\Users\aurel\Desktop\video_oiseaux_cam2\faster_rcnn_inception_resnet_v2_1024x1024_coco17_tpu-8\saved_model'
# load model from path
model= tf.saved_model.load(model_dir)


In [ ]:
# read image and preprocess
img = cv2.cvtColor(img0, cv2.COLOR_BGR2RGB)
img = img[200:-20, 200:, :]
input_tensor = np.expand_dims(img, 0)

# predict from model
resp = model(input_tensor)

In [ ]:
class_names = {
    1:'person',
    3:'car',
    6:'bus',
    8:'truck',
    9:'boat',
    16:'bird'
}

h, w, z = img.shape

In [ ]:
# iterate over boxes, class_index and score list
for boxes, classes, scores in zip(resp['detection_boxes'].numpy(), resp['detection_classes'], resp['detection_scores'].numpy()):
    for box, cls, score in zip(boxes, classes, scores):
        if score > 0.05: 
            ymin = int(box[0] * h)
            xmin = int(box[1] * w)
            ymax = int(box[2] * h)
            xmax = int(box[3] * w)
            cv2.putText(img, class_names[int(cls)], (xmin, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (128, 0, 128), 1)

In [ ]:
# convert back to bgr and save image
cv2.imwrite(r'C:\Users\aurel\Desktop\video_oiseaux_cam2\output_zoom.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

In [ ]:
plt.imshow(img)